## Imports

In [1]:
import pandas as pd
import folium
import numpy as np
import os
import googlemaps
import ast

## Data Read-in

In [2]:
df = pd.read_csv("project_sites_expanded.csv")

In [3]:
df

,Site,Lot Count,Address,BBL,Lot Area,Zoning Change,Existing Building & Use,No Action,With Action,Increment 1,Increment 2,Increment 3,Increment 4,Increment 5,Increment 6
0,PROJECTED SITE 1,2,539/543 8th Avenue,"1007610031, 1007610032","5,404 sf",M1-6D to M1-9A/R12,4-5 story buildings with ground floor retail a...,Continuation of existing use,"87,615 zsf of residential (103 units), 215’ in...",+ 103 units,+ 20 Inclusionary Housing units,"+ 4,616 zsf of local retail","- 70,516 zsf of office",NaN,NaN
1,PROJECTED SITE 2,1,253 W 28 St,1007780007,"6,016 sf",M1-6D to M1-9A/R12,8 story mixed-use building with ground floor r...,Continuation of existing use,"94,540 zsf of residential (111 units), 415’ in...",+ 110 units,+ 28 Inclusionary Housing units,"+ 6,332 zsf of local retail","- 25,398 zsf of office",NaN,NaN
2,PROJECTED SITE 3,3,210/212 W 29/207 28 St,"1007780034, 1007780046, 1007780047","15,541 sf",M1-6 to M1-8A/R12,Three 4-9 story loft buildings with ground-flo...,Continuation of existing use,"256,966 zsf of residential (302 units), 345’ i...",+ 302 units,+ 76 Inclusionary Housing units,"+ 22,966 zsf of local retail","- 70,516 zsf of office",NaN,NaN
3,PROJECTED SITE 4,1,253 W 29th Street,1007790008,"4,678 sf",M1-6D to M1-9A/R12,Surface parking,Continuation of existing use,"75,873 zsf of residential (89 units), 325’ in ...",+ 89 units,+ 23 Inclusionary Housing units,"+ 8,140 zsf of local retail",NaN,,NaN
4,PROJECTED SITE 5,1,250 W 30th Street,1007790069,"5,643 sf",M1-6D to M1-9A/R12,3-story office building,Continuation of existing use,"90,690 zsf of residential (106 units), 315’ in...",+ 106 units,+ 27 Inclusionary Housing units,"+ 9,819 zsf of local retail","- 12,886 zsf of office",NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56,PROJECTED SITE 58,2,18/20 W 38 St,"1008390058, 1008390059","4,974 sf",M1-6 to M1-9A/R12,Two 4-5 story buildings with ground floor reta...,Continuation of existing use,"80,357 zsf of residential (94 units), 325’ in ...",+ 92 units,+ 24 Inclusionary Housing units,"+ 4,956 zsf of local retail","- 9,635 zsf of office",NaN,NaN
57,PROJECTED SITE 59,2,38/40 W 39 St,"1008400069, 1008400070","5,005 sf",M1-6 to M1-9A/R12,Two 5 story buildings with ground-floor retail...,Continuation of existing use,"80,790 zsf of residential (95 units), 325’ in ...",+ 95 units,+ 24 Inclusionary Housing units,"+ 5,055 zsf of local retail","- 13,268 zsf of office",NaN,NaN
58,PROJECTED SITE 60,1,35 W 38 St,1008400021,"3,859 sf",M1-6 to M1-9A/R12,I 5 story buildings with ground-floor retail a...,Continuation of existing use,"62,578 zsf of residential (73 units), 305’ in ...",+ 73 units,+ 19 Inclusionary Housing units,"+ 3,885 zsf of local retail","- 7,656 zsf of office",NaN,NaN
59,PROJECTED SITE 61,1,20 W 39 St,1008400060,"4,341 sf",M1-6 to M1-9A/R12,One 3 story structure occupied by a single cab...,Continuation of existing use,"69,649 zsf of residential (81 units), 325’ in ...",+ 81 units,+ 21 Inclusionary Housing units,"- 7,425 zsf of local retail",NaN,NaN,NaN


In [4]:
df['full_address'] = df['Address'] + ' New York, NY'

In [5]:
%store -r google_maps_API_Key
gmaps_key = googlemaps.Client(key=google_maps_API_Key)

In [6]:
# Define the geocode function
def geocode(add):
    g = gmaps_key.geocode(add)
    if g:
        lat = g[0]["geometry"]["location"]["lat"]
        lng = g[0]["geometry"]["location"]["lng"]
        return (lat, lng)
    else:
        return None

# Apply geocoding to the 'geo_address' column and store the results in 'geocoded' column
df['geocoded'] = df['full_address'].apply(geocode)

In [7]:
df['geocoded'] = df['geocoded'].astype(str)
df[['lat', 'lon']] = df['geocoded'].apply(lambda x: (None, None) if x == 'None' else x.strip('()').split(', ', 1)).apply(pd.Series)
df['lat'] = df['lat'].astype(float)
df['lon'] = df['lon'].astype(float)

In [10]:
df = df.rename(columns={'Increment 1':'Units','Increment 2':'Inclusionary Housing units',
                  'Increment 3':'Local Retail','Increment 4':'Office','Increment 5':'Warehouse',
                  'Increment 6':'Industrial'})

In [18]:
df.geocoded.isna()

geocoded
False    61
Name: count, dtype: int64

In [16]:
import folium
from folium.plugins import MarkerCluster
import pandas as pd
import textwrap
import ast  # Needed for literal_eval

def tooltip_html(df, row_index, max_chars=30):
    """
    Generate HTML content for the tooltip with line breaks at word boundaries.
    
    Parameters:
    - df (pd.DataFrame): DataFrame containing the property data.
    - row_index (int): The index of the row in the DataFrame.
    - max_chars (int): Maximum number of characters per line.
    
    Returns:
    - str: HTML string for the tooltip content.
    """
    # Retrieve the values from the DataFrame
    address = df.at[row_index, 'Address']
    lot_area = df.at[row_index, 'Lot Area']
    z_change = df.at[row_index, 'Zoning Change']
    desc = df.at[row_index, 'Existing Building & Use']
    no_action = df.at[row_index, 'No Action']
    with_action = df.at[row_index, 'With Action']
    units = df.at[row_index, 'Units']
    incl_housing = df.at[row_index, 'Inclusionary Housing units']
    local_retail = df.at[row_index, 'Local Retail']
    office = df.at[row_index, 'Office']
    warehouse = df.at[row_index, 'Warehouse']
    industrial = df.at[row_index, 'Industrial']
    
    # Convert each field to a string or default to 'N/A' if missing
    address_str = str(address).strip() if pd.notnull(address) else 'N/A'
    lot_area_str = str(lot_area).strip() if pd.notnull(lot_area) else 'N/A'
    z_change_str = str(z_change).strip() if pd.notnull(z_change) else 'N/A'
    desc_str = str(desc).strip() if pd.notnull(desc) else 'N/A'
    no_action_str = str(no_action).strip() if pd.notnull(no_action) else 'N/A'
    with_action_str = str(with_action).strip() if pd.notnull(with_action) else 'N/A'
    units_str = str(units).strip() if pd.notnull(units) else 'N/A'
    incl_housing_str = str(incl_housing).strip() if pd.notnull(incl_housing) else 'N/A'
    local_retail_str = str(local_retail).strip() if pd.notnull(local_retail) else 'N/A'
    office_str = str(office).strip() if pd.notnull(office) else 'N/A'
    warehouse_str = str(warehouse).strip() if pd.notnull(warehouse) else 'N/A'
    industrial_str = str(industrial).strip() if pd.notnull(industrial) else 'N/A'
    
    # Wrap the address to ensure it fits nicely in the tooltip
    wrapped_address = '<br>'.join(textwrap.wrap(address_str, width=max_chars, break_long_words=False))
    
    # Construct the tooltip HTML content with labels for each field
    tooltip_content = f"""
    <div class="popup-content">
        <strong>Address:</strong> {wrapped_address}<br>
        <strong>Lot Area:</strong> {lot_area_str}<br>
        <strong>Zoning Change:</strong> {z_change_str}<br>
        <strong>Existing Building &amp; Use:</strong> {desc_str}<br>
        <strong>No Action:</strong> {no_action_str}<br>
        <strong>With Action:</strong> {with_action_str}<br>
        <strong>Units:</strong> {units_str}<br>
        <strong>Inclusionary Housing:</strong> {incl_housing_str}<br>
        <strong>Local Retail:</strong> {local_retail_str}<br>
        <strong>Office:</strong> {office_str}<br>
        <strong>Warehouse:</strong> {warehouse_str}<br>
        <strong>Industrial:</strong> {industrial_str}
    </div>
    """
    return tooltip_content

# Assuming df is already defined and loaded with your data
# Initialize the map centered around the first geocoded point
map_center = ast.literal_eval(df['geocoded'].iloc[0])
m = folium.Map(location=map_center, zoom_start=12)

# Add the custom Mapbox tile layer
folium.TileLayer(
    tiles='https://api.mapbox.com/styles/v1/mapbox/streets-v11/tiles/256/{z}/{x}/{y}@2x?access_token=pk.eyJ1IjoidHJkZGF0YSIsImEiOiJjamc2bTc2YmUxY2F3MnZxZGh2amR2MTY5In0.QlOWqB-yQNrNlXD0KQ9IvQ',
    attr='Mapbox',
    name='Streets',
    overlay=True,
    control=False,
    show=False,
    min_zoom=1,
    max_zoom=20
).add_to(m)

# Add custom CSS to style the tooltips
custom_css = """
<style>
    .popup-content {
        max-width: 600px; /* Adjusts the maximum width of the tooltip */
        font-size: 12px;   /* Adjusts the font size */
        word-wrap: break-word; /* Ensures long words wrap to the next line */
    }
    .leaflet-tooltip {
        background-color: white;
        border: 1px solid gray;
        border-radius: 3px;
        padding: 5px;
    }
</style>
"""
m.get_root().html.add_child(folium.Element(custom_css))

# Create and add the title to the map
title_html = '''
    <h3 align="center" style="font-size:16px"><b>Midtown South Rezoning</b></h3>
'''
m.get_root().html.add_child(folium.Element(title_html))

# Initialize a MarkerCluster to handle multiple markers efficiently
marker_cluster = MarkerCluster().add_to(m)

# Loop through the DataFrame to create markers
for i, row in df.iterrows():
    # Create tooltip HTML content with all fields
    tooltip_content = tooltip_html(df, i)
    
    # Create a CircleMarker for each row
    folium.CircleMarker(
        location=[row['lat'], row['lon']],
        tooltip=folium.Tooltip(tooltip_content, sticky=True),
        color='red',
        fill=True,
        fill_color='red',
        radius=9,
        weight=1,        # Thickness of the circle border
        opacity=1,       # Opacity of the circle border
        fill_opacity=1   # Opacity of the fill
    ).add_to(marker_cluster)

# Add layer control to toggle different layers
folium.LayerControl().add_to(m)

# Display the map
m


In [24]:
m.save('index.html')

In [25]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/MarkNussbaumMap
